In [1]:
import pandas as pd
from glob import glob
import json
import os

In [53]:
def get_filelst(SAVE_PATH):
    return list(set([i.replace("_0_shot.json","") for i in os.listdir(SAVE_PATH)]))

# results/korean_origin_bench/20b/00_shot.json
def _get_metric_name(v):
    metrics = ['f1', 'macro_f1', 'acc_norm', 'acc']
    for m in metrics:
        if v.get(m):
            return {
                'metric': m,
                'value': v[m],
            }

def get_df_klue(SAVE_PATH, model_name=''):
    
    data = []
    for i in ['0', '5', '10', '50']:
        shot = f'{SAVE_PATH}/{model_name}_{i}_shot.json'
        print(shot)
        try:
            data.append(
                {
                    f"{k} ({_get_metric_name(v)['metric']})": _get_metric_name(v)['value']
                    for k, v in json.load(open(shot))['results'].items()
                }
            )
        except FileNotFoundError:
            pass
    df = pd.DataFrame(data, index=[0, 5, 10, 50][:len(data)]).T
    print(data)
    # print(df.to_markdown())
    return df

In [54]:
SAVE_PATH = '../data/result'
filelst = get_filelst(SAVE_PATH)
df = get_df_klue(SAVE_PATH,model_name=filelst[0])

../data/result/aipart_chatjob_sft_from_Synatra-Mixtral-8x7B-v0.1_0_shot.json
../data/result/aipart_chatjob_sft_from_Synatra-Mixtral-8x7B-v0.1_5_shot.json
../data/result/aipart_chatjob_sft_from_Synatra-Mixtral-8x7B-v0.1_10_shot.json
../data/result/aipart_chatjob_sft_from_Synatra-Mixtral-8x7B-v0.1_50_shot.json
[{'kobest_boolq (macro_f1)': 0.35461338879835325, 'kobest_copa (macro_f1)': 0.6577316616227121, 'kobest_hellaswag (macro_f1)': 0.4491881401493295, 'kobest_sentineg (macro_f1)': 0.5252590348936292, 'kohatespeech (macro_f1)': 0.4770029338749717, 'kohatespeech_apeach (macro_f1)': 0.33851558894747785, 'kohatespeech_gen_bias (macro_f1)': 0.12453531598513011, 'korunsmile (f1)': 0.30648331029034165, 'nsmc (acc)': 0.56638, 'pawsx_ko (acc)': 0.539}]


In [55]:
df

,0
kobest_boolq (macro_f1),0.354613
kobest_copa (macro_f1),0.657732
kobest_hellaswag (macro_f1),0.449188
kobest_sentineg (macro_f1),0.525259
kohatespeech (macro_f1),0.477003
kohatespeech_apeach (macro_f1),0.338516
kohatespeech_gen_bias (macro_f1),0.124535
korunsmile (f1),0.306483
nsmc (acc),0.566380
pawsx_ko (acc),0.539000
